#Step 01. Install All Dependencies

This installs Apache Spark 2.3.3, Java 8, Findspark library that makes it easy for Python to work on the given Big Data.

In [1]:
import os
#OpenJDK Dependencies for Spark
os.system('apt-get install openjdk-8-jdk-headless -qq > /dev/null')

#Download Apache Spark
os.system('wget -q http://apache.osuosl.org/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz') 

#Apache Spark and Hadoop Unzip
os.system('tar xf spark-2.3.3-bin-hadoop2.7.tgz')

#FindSpark Install
os.system('pip install -q findspark')

0

# Step 02. Set Environment Variables
Set the locations where Spark and Java are installed based on your installation configuration. Double check before you proceed.

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

# Step 03. ELT - Load the Data: Mega Cloud Access
This is an alternative approach to load datasets from already stored in [**Mega Cloud**](https://mega.nz) cloud repository. You need to install the necessary packages and put the link URL of cloud to load the file from cloud directly.

In [4]:
import os
os.system('git clone https://github.com/jeroenmeulenaar/python3-mega.git')
os.chdir('python3-mega')
os.system('pip install -r requirements.txt')

0

# Step 04. ELT - Load the Data: Read Uploaded Dataset
In this approach you can directly load the uploaded dataset downloaded fro Mega Cloud Infrastructure

In [0]:
from mega import Mega
os.chdir('../')
m_revenue = Mega.from_ephemeral()
m_revenue.download_from_url('https://mega.nz/#!Uwo2QKAT!TBB6FxWEA9-2qYdlMKmVhiLtJfVZp-jnxcQzZ-I7Ja0')

# Step 05. Start a Spark Session
This basic code will prepare to start a Spark session.

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-datathon19-easy02').master("local[*]").getOrCreate()

# Step 06. Exploration - Data Schema View
Now let's load the DataFrame and see the schema view of the Spark dataset

In [7]:
df = spark.read.csv('devices.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- imei_tac: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- os_name: string (nullable = true)
 |-- os_vendor: string (nullable = true)



# Step 07. Exploration - Row Count
Now since all the rows are here string/text formatted there is no meaning of running statistical method over the values of these columns. But we need to know the number of rows. We'll grab total number of entries to have an overview of data

In [8]:
print("Total Rows: ")
df.count()

Total Rows: 


20463280

# Step 08. Exploration - Total Unique Row Count
Now we'll grab total number of unique entries or unique row count of the Spark dataset to have an overview of duplicate data

In [9]:
print("Unique Rows: ")
df.distinct().count()

Unique Rows: 


20463280

# Step 09. Exploration - Reviewing the NULL values in each column
Since the total row count and unique row count are same, it means there is no duplicate rows in the table. Now we'll grab the count of NULL values per column to check whether any missing values is there or not.

In [10]:
import pyspark.sql.functions as F
df_agg = df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns])
df_agg.show()

+------+--------+----------+----------+-------+---------+
|msisdn|imei_tac|brand_name|model_name|os_name|os_vendor|
+------+--------+----------+----------+-------+---------+
|     0|       0|    256304|    256304|5685204|  5540476|
+------+--------+----------+----------+-------+---------+



# Step 10. Exploration - Filtering the NULL values rows of Model entries
Roughly `1.25%` data are `NULL` values from the table. Since this is neglitible compare to the original row count, we will now filter the dataset to remove all `NULL` values of `model_name` column.

In [0]:
df2 = df.filter(df.model_name. isNotNull())

# Step 11. Implementation - Run the SQL Command
Now since we got the idea that there is no NULL values and we optimises the dual SIM enabled mobile set rows, we can straightly go for executing SQL command to get the desired outcome. As a part of optimisation, we can drop of the column week_number as this is not relevant to this problem.

In [15]:
from pyspark.sql.functions import desc
Easy02 = df2.groupBy('model_name').agg({'model_name':'count'}).sort(desc("count(model_name)"))
Easy02 = Easy02.withColumnRenamed("count(model_name)", "Total Bought")
Easy02 = Easy02.withColumnRenamed("model_name", "Model")
Easy02.show(n=5, truncate=False)

+-------------------------------------+------------+
|Model                                |Total Bought|
+-------------------------------------+------------+
|GURU MUSIC II (SM-B310E)             |406457      |
|GALAXY J2 (SM-J200H)                 |225834      |
|GALAXY GRAND PRIME DUOS (SM-G532F DS)|220640      |
|105, 1050 (RM-908)                   |194412      |
|SYMPHONY B12, D115                   |191265      |
+-------------------------------------+------------+
only showing top 5 rows

